# Collaborative filters' comparison

In [6]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from source.models.collabFilter import CollaborativeFilter
from source.models.collabFilter_UserUser import CollaborativeFilter_UserUser
from source.models.collabFilter_ItemItem import CollaborativeFilter_ItemItem

import source.utils.utils as utils

In [7]:
data = pd.read_csv('../data/data_clean.csv')
train_data = pd.read_csv('../data/train_data.csv')
test_data = pd.read_csv('../data/test_data.csv')

In [8]:
U = train_data.pivot_table(index='tessera', columns='cod_prod', values='value') # or index='id_sc'

In [9]:
cfUser : CollaborativeFilter =  CollaborativeFilter_UserUser(N = 100)
cfUser.fit(U)

In [10]:
cfItem : CollaborativeFilter =  CollaborativeFilter_ItemItem(N = 100)
cfItem.fit(U)

In [11]:
predicted_ratings_user, predicted_ratings_item = utils.parallel_predict(cfUser, cfItem, test_data)

  0%|          | 0/286 [00:00<?, ?it/s]/Users/daniele/Documents/Università/terzoAnno/primoSemestre/SocialMediaDataAnalysis/Progetto/ShopSmart/source/models/collabFilter_UserUser.py:9: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))
  0%|          | 0/286 [00:00<?, ?it/s]/Users/daniele/Documents/Università/terzoAnno/primoSemestre/SocialMediaDataAnalysis/Progetto/ShopSmart/source/models/collabFilter_ItemItem.py:9: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))
  0%|          | 1/286 [00:03<14:55,  3.14s/it]/Users/daniele/Documents/Università/terzoAnno/primoSemestre/SocialMediaDataAnalysis/Progetto/ShopSmart/source/models/collabFilter_UserUser.py:9: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))
  4%|▍         | 12/286 [00:00<00:08, 33.08i

In [12]:
meanVelue = test_data['value'].mean()
print(f"Mean Value: {meanVelue}")

Mean Value: 0.005219045660222112


In [13]:
mae_score_user = utils.mae(test_data['value'], predicted_ratings_user)

In [14]:
mae_score_item = utils.mae(test_data['value'], predicted_ratings_item)

In [15]:
xi = U.index[0] # for example

if mae_score_user < mae_score_item:
    print(f"User-User Collaborative Filtering has a better performance with a MAE of {mae_score_user}")
    recommendations = cfUser.recommend(xi, 5)
else:
    print(f"Item-Item Collaborative Filtering has a better performance with a MAE of {mae_score_item}")
    recommendations = cfItem.recommend(xi, 5)

Item-Item Collaborative Filtering has a better performance with a MAE of 0.00994683681292687


/Users/daniele/Documents/Università/terzoAnno/primoSemestre/SocialMediaDataAnalysis/Progetto/ShopSmart/source/models/collabFilter_ItemItem.py:9: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))
/Users/daniele/Documents/Università/terzoAnno/primoSemestre/SocialMediaDataAnalysis/Progetto/ShopSmart/source/models/collabFilter_ItemItem.py:21: RuntimeWarning: invalid value encountered in scalar divide
  predicted_rating = (self.U.loc[xi, similar_items] * selected_similarities).sum() / selected_similarities.sum()


In [16]:
data['cod_prod'] = data['cod_prod'].astype(str)

In [17]:
descriptions = []
for item, _ in recommendations:
    item_str = str(item)
    if not data.loc[data['cod_prod'] == item_str, 'descr_prod'].empty:
        descriptions.append(data.loc[data['cod_prod'] == item_str, 'descr_prod'].values[0])
    else:
        print(f"Item {item_str} non trovato nel DataFrame")

recommendations_str = "\n".join(descriptions)
print(f"Top 5 recommendations for user {xi}:\n{recommendations_str}")


Top 5 recommendations for user 248401194304.0:
1000062       COCCOLINO BLU PRIM.LT3 40LAV(P
1000254       MB GALLETTI GR.800            
1000279       PAVESI GOCCIOLE CLASSICHE 500G
1000347        MB PANE GRANO DURO GR.400    
1000325       BARILLA SUGO AMATRICIANA GR400
